# 移动平均线理论
捕捉趋势最普遍的方法是“移动平均线”，根据求平均的方式不同，可以分为：

## 1. 简单移动平均 SMA

 SMA，将一组股价相加，再除以股价的个数。
 例如，求5日的平均数，先day1到day5的价格之和后，除以5.得到第5天的简单平均数。第6天的简单平均数以此类推。

$$
SMA_{t=5} = \frac{p1 + p2 + p3 + p4 + p5}{5}
$$

所以，SMA中期数的选择，从以下几方面考虑：
1. 事件发展的周期性；
2. 对趋势平均性的要求；
3. 对趋势反映近期变化敏感程度的要求。

## 2. 加权移动平均数 WMA

WMA，是在SMA的基础上对每个平均价值增加权重。在预测股价时，不同时期的股价数据具有不同的代表性。为了表示其代表性，在SMA基础上赋予一定的权重。

例如，5日加权移动平均值

$$
\begin{aligned}
& WMA_{t=5} = w_{1}p_{1} + w_{2}p_{2} + w_{3}p_{3} + w_{4}p_{4} + w_{5}p_{5} \\

& w_{1} , w_{5} 为股价数据的权重，之和等于1.\\
\end{aligned}
$$

通过对数据加权，将当前时间离的越近的数据越具有代表性，越远的数据代表性越低。

## 3. 指数加权移动平均数EWMA(或EXPMA)

指数加权平均数EWMA 是一种特别的加权移动平均。
$$
\begin{aligned}
& p_{t} 表示股票第t期的价格，从第k期开始计算股价的加权移动平均数。\\
& \alpha 和 1 - \alpha 表示权重 \\
& EWMA_{t=k} = \frac{p1 + p2 + ... + pk}{k} \\
& EWMA_{t=k+1} = p_{k+1} * \alpha + EWMA_{t=k} * (1 -\alpha)\\
& EWMA_{t=k+2} = p_{k+2} * \alpha + EWMA_{t=k+1} * (1 - \alpha)\\
& ......\\
& \\
& 最终推导，当 t 大于等于 k + 1时，\\
& EWMA_{t} = \alpha * [p_{t} + p_{t-1} * (1 - \alpha ) + ... + p_{k+1}(1-\alpha)^(t-k-1) ] \\
&      + EWMA_{t=k} * (1 - \alpha )^(t-k)

\end{aligned}
$$

# 双均线交易策略
## 策略规则

双均线交易策略，是利用长短期均线的相对关系来判断未来趋势。策略制定过程：

1. 计算短期均线 和 长期均线
2. 当短期均线从下向上穿过长期均线时，释放出买入信号；
3. 当长期均线从上向下穿过短期均线时，释放出卖出信号。

在这里，601318 中国平安 为例，计算其2022-01-01 至 2023-01-01期间每日行情的MA5（短期均线）、MA20（长期均线）。



In [1]:
import sys
sys.path.append(r'/Users/paul/DSWorkspace/001量化策略/002-Project项目/lleasy')

# 1. 数据准备
from lleasy.database import SqliteDatabase as sqlitedb
from lleasy.object import TradeData, BarData
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

## 数据准备

In [5]:
# 获取数据
# 标的：601318 中国平安
# 时间：2022-01-01 至 2023-01-01
# 频率：日行情
#获取sqlite链接
db = sqlitedb()
bars = db.get_symbol_bars_bySEDate('601318','2022-01-01','2023-01-01')
bars=bars.set_index(['datetime'])
bars


2023-08-01 17:31:47,739 DEBUG sqlalchemy.pool.impl.QueuePool Created new connection <sqlite3.Connection object at 0x12562f840>
2023-08-01 17:31:47,739 DEBUG sqlalchemy.pool.impl.QueuePool Connection <sqlite3.Connection object at 0x12562f840> checked out from pool
2023-08-01 17:31:47,740 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-01 17:31:47,742 INFO sqlalchemy.engine.Engine SELECT llbardata.id AS llbardata_id, llbardata.symbol AS llbardata_symbol, llbardata.interval AS llbardata_interval, llbardata.open AS llbardata_open, llbardata.close AS llbardata_close, llbardata.high AS llbardata_high, llbardata.low AS llbardata_low, llbardata.vol AS llbardata_vol, llbardata.turnover AS llbardata_turnover, llbardata.asset AS llbardata_asset, llbardata.adjust AS llbardata_adjust, llbardata.datetime AS llbardata_datetime 
FROM llbardata 
WHERE llbardata.symbol = ? AND llbardata.datetime >= ? AND llbardata.datetime <= ?
2023-08-01 17:31:47,743 INFO sqlalchemy.engine.Engine [generated in 0

,symbol,open,close,high,low
datetime,,,,,
2022-01-04,601318,50.400,51.000,51.060,50.100
2022-01-05,601318,51.110,52.070,52.400,50.970
2022-01-06,601318,51.990,51.300,52.020,51.220
2022-01-07,601318,51.490,52.940,53.330,51.450
2022-01-10,601318,52.990,53.080,53.560,52.500
...,...,...,...,...,...
2022-12-26,601318,45.640,44.660,45.880,44.660
2022-12-27,601318,45.270,45.580,45.950,45.030
2022-12-28,601318,45.430,46.280,46.450,45.300


## 编制策略

为更好演示，在这里将对SMA、WMA、EWMA分别定义相应的独立函数。

In [14]:

def sma(data:pd.DataFrame, period:int) -> pd.DataFrame:
    lable_sma = 'sma' + str(period)
    data[lable_sma] = data['close'].rolling(period).mean()
    data[lable_sma] = data[lable_sma].astype('float')
    return data

def wma():
    pass

def ewma():
    pass

## 执行策略

In [38]:
# 计算SMA5、SMA20移动平均值
sma_bars = bars
sma_bars = sma(sma_bars,5)
sma_bars = sma(sma_bars,20)

sma_bars['close'] = sma_bars['close'].astype('float')
#删除NA数据
sma_bars.dropna(inplace=True)


In [36]:
# 图形化显示 SMA5、SMA20
import plotly.graph_objects as go

trace1 = go.Scatter(
    x=sma_bars.index,
    y=sma_bars['sma5'],
    mode='lines',
    name='sma5',
    marker={
        'size':8,
#        'color':'rgba(102,198,147,0.7)',
    }
)
trace2 = go.Scatter(
    x=sma_bars.index,
    y=sma_bars['sma20'],
    mode='lines',
    name='sma20',
    marker={
        'size':8,
#        'color':'rgba(252,108,117,1)'
    }
)

trace3 = go.Scatter(
    x=sma_bars.index,
    y=sma_bars['close'],
    mode='lines',
    name='close'
)

fig = go.Figure(data=[trace1,trace2,trace3])
fig.show()

In [39]:
# 1. 找出买卖位置
#当短期均线从下向上穿过长期均线时，释放出买入信号；sma5>sma20 ，标记为1
#当长期均线从上向下穿过短期均线时，释放出卖出信号。sma5<sma20 ，标记为-1
sma_bars['position'] = np.where(sma_bars['sma5'] > sma_bars['sma20'],1,-1)
sma_bars

,symbol,open,close,high,low,sma5,sma20,position
datetime,,,,,,,,
2022-02-07,601318,51.340,51.98,52.470,50.700,51.154,51.9310,-1
2022-02-08,601318,51.760,53.00,53.030,51.720,51.514,52.0310,-1
2022-02-09,601318,53.240,53.04,54.000,52.880,51.756,52.0795,-1
2022-02-10,601318,53.180,54.18,54.180,52.800,52.434,52.2235,1
2022-02-11,601318,54.350,55.59,56.230,54.080,53.558,52.3560,1
...,...,...,...,...,...,...,...,...
2022-12-26,601318,45.640,44.66,45.880,44.660,45.274,46.0290,-1
2022-12-27,601318,45.270,45.58,45.950,45.030,45.404,46.0690,-1
2022-12-28,601318,45.430,46.28,46.450,45.300,45.608,46.1355,-1


## 策略回归

# 总结